In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

X_VER = 412

L_VER = 514
USE = 'orders'

# USE 0,1,2,3
PART = 0

In [2]:
import pandas as pd, cudf, numpy as np, gc
import matplotlib.pyplot as plt

In [3]:
users = pd.read_parquet(f'../../data/infer_with_features/train_v175_LB_{PART}_X.pqt')

# XGBoost

In [4]:
FEATURES = users.columns[2:]
TARS = [USE]
FEATURES = [f for f in FEATURES if f not in TARS]
print(len(FEATURES))
print( FEATURES)
print(TARS)

223
['pos', 'wgt', 'score_1', 'score_3', 'score_4', 'score_5', 'score_6', 'score_7', 'score_10', 'score_11', 'score_12', 'score_13', 'score_30', 'score_14', 'score_15', 'score_16', 'score_17', 'score_18', 'score_19', 'score_20', 'score_21', 'score_22', 'score_23', 'score_24', 'score_25', 'score_26', 'buy_ratio_5', 'count_item_5', 'count_user_5', 'repeat_5', 'buy_ratio_std_5', 'item_day_std_5', 'user_day_std_5', 'buy_ratio_0_6', 'buy_ratio_1_6', 'buy_ratio_2_6', 'buy_ratio_3_6', 'buy_ratio_4_6', 'buy_ratio_5_6', 'buy_ratio_6_6', 'count_item_0_6', 'count_item_1_6', 'count_item_2_6', 'count_item_3_6', 'count_item_4_6', 'count_item_5_6', 'count_item_6_6', 'count_user_0_6', 'count_user_1_6', 'count_user_2_6', 'count_user_3_6', 'count_user_4_6', 'count_user_5_6', 'count_user_6_6', 'item_day_std_6', 'user_day_std_6', 'buy_ratio', 'count_item', 'count_user', 'hour_mean', 'day_mean', 'repeat', 'buy_ratio_std', 'hour_std', 'day_std', 'buy_ratio3', 'count_item3', 'count_user3', 'hour_mean3', 'day

In [5]:
%%time
print('BEFORE', users.shape )
users = users.sort_values('user').reset_index(drop=True)
print('AFTER', users.shape )

BEFORE (41795000, 225)
AFTER (41795000, 225)
CPU times: user 22.3 s, sys: 43.9 s, total: 1min 6s
Wall time: 1min


In [8]:
oof = np.zeros(len(users))

In [ ]:
%%time
from cuml import ForestInference
FOLDS = 5

for fold in range(FOLDS):
    print('#'*25)
    print('### Fold',fold+1)
    print('#'*25)
    
    print(f'=> predicting XGB {X_VER}...')
    #model = xgb.Booster()
    #model.load_model(f'../../data/models/XGB_fold{fold}_{USE}_v{X_VER}.xgb')
    #model.set_param({'predictor': 'gpu_predictor'})
    #model.set_param({'gpu_id': 0})
    model = ForestInference.load(f'../../data/models/XGB_fold{fold}_{USE}_v{X_VER}.xgb')
    
    # INFER OOF FOLD K
    print('inferring 1of4...')
    X_valid = users[FEATURES].iloc[:len(users)//4]
    #dvalid = xgb.DMatrix(data=X_valid) 
    oof_preds = model.predict(X_valid)
    oof[:len(users)//4] += oof_preds/FOLDS
    del X_valid#, dvalid
    gc.collect()
    
    # INFER OOF FOLD K
    print('inferring 2of4...')
    X_valid = users[FEATURES].iloc[len(users)//4:(2*len(users))//4]
    #dvalid = xgb.DMatrix(data=X_valid) 
    oof_preds = model.predict(X_valid)
    oof[len(users)//4:(2*len(users))//4] += oof_preds/FOLDS
    del X_valid#, dvalid
    gc.collect()
    
    # INFER OOF FOLD K
    print('inferring 3of4...')
    X_valid = users[FEATURES].iloc[(2*len(users))//4:(3*len(users))//4]
    #dvalid = xgb.DMatrix(data=X_valid) 
    oof_preds = model.predict(X_valid)
    oof[(2*len(users))//4:(3*len(users))//4] += oof_preds/FOLDS
    del X_valid#, dvalid
    gc.collect()
    
    print('inferring 4of4...')
    X_valid = users[FEATURES].iloc[(3*len(users))//4:]
    #dvalid = xgb.DMatrix(data=X_valid) 
    oof_preds = model.predict(X_valid)
    oof[(3*len(users))//4:] += oof_preds/FOLDS

    del X_valid, model#, dvalid 
    gc.collect()

In [10]:
%%time
if 0:
    for c in users.columns[2:]:
        print(c,', ',end='')
        if str(users[c].dtype)=='float32':
            users[c] = users[c].astype('float16')
        if str(users[c].dtype)=='int32':
            users[c] = users[c].astype('int16')

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 15 µs


In [11]:
%%time
oof2 = np.zeros(len(users))

if 0:
    for fold in range(5):
        print('#'*25)
        print('### Fold',fold+1)
        print('#'*25)

        print(f'=> predicting LGB {L_VER}...')
        model = lgb.Booster( model_file=f'../../jan-22-23-otto/ranker_models/LGB_fold{fold}_{USE}_v{L_VER}.lgb' )
        oof_preds = model.predict(users[FEATURES2])
        oof2 += oof_preds/FOLDS

        del model
        gc.collect()

CPU times: user 275 µs, sys: 42 µs, total: 317 µs
Wall time: 274 µs


# Load Preds

In [12]:
if 0:
    oof = np.load(f'/raid/Kaggle/otto/pred10/pred{X_VER}_v{PART}.npy')
    oof2 = np.load(f'/raid/Kaggle/otto/pred10/pred{L_VER}_v{PART}.npy')

# Make Sub

In [13]:
sub = users[['user','item']].copy()

sub['p1'] = oof
#sub['p2'] = oof2

print( sub.shape )
sub.head()

(41795000, 3)


,user,item,p1
0,12899779,59625,3.374460
1,12899779,16620,-3.750089
2,12899779,158571,-3.586417
3,12899779,1425325,-6.246592
4,12899779,1167790,-5.409854


In [14]:
%%time
if 0:
    sub['p1'] = sub.groupby('user').p1.rank()
    #sub['p2'] = sub.groupby('user').p2.rank()

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 15.5 µs


In [15]:
%%time
sub['p'] = sub.p1

sub = sub.sort_values(['user','p'],ascending=[True,False])
print( sub.shape )
sub.head()

(41795000, 4)
CPU times: user 41.1 s, sys: 3.18 s, total: 44.3 s
Wall time: 44.2 s


,user,item,p1,p
0,12899779,59625,3.374460,3.374460
76,12899779,731692,1.599687,1.599687
94,12899779,448688,-0.239109,-0.239109
69,12899779,1790770,-0.334835,-0.334835
68,12899779,25820,-0.433626,-0.433626


In [16]:
sub = sub.reset_index(drop=True)

In [17]:
sub['n'] = sub.groupby('user').item.cumcount().astype('int8')

In [18]:
sub.head()

,user,item,p1,p,n
0,12899779,59625,3.374460,3.374460,0
1,12899779,731692,1.599687,1.599687,1
2,12899779,448688,-0.239109,-0.239109,2
3,12899779,1790770,-0.334835,-0.334835,3
4,12899779,25820,-0.433626,-0.433626,4


In [19]:
sub = sub.loc[sub.n<20]
print( sub.shape )

(8359000, 5)


In [20]:
%%time
sub2 = sub.groupby('user').item.apply(list)

CPU times: user 7.14 s, sys: 176 ms, total: 7.32 s
Wall time: 7.31 s


In [21]:
sub2 = sub2.to_frame().reset_index()

In [22]:
%%time
sub2.item = sub2.item.apply(lambda x: " ".join(map(str,x)))

CPU times: user 1.59 s, sys: 68.4 ms, total: 1.66 s
Wall time: 1.66 s


In [23]:
sub2.columns = ['session_type','labels']

In [24]:
sub2.session_type = sub2.session_type.astype('str')+ f'_{USE}'

In [25]:
sub2.head()

,session_type,labels
0,12899779_orders,59625 731692 448688 1790770 25820 475447 17002...
1,12899780_orders,1142000 582732 736515 973453 1758603 487136 59...
2,12899781_orders,918667 199008 1681537 1836671 1628918 759436 1...
3,12899782_orders,1711180 127404 562753 1344773 829180 740494 98...
4,12899783_orders,1817895 1811433 1729553 573058 1476106 1216820...


In [26]:
sub2.to_csv(f'../../data/submission_parts/submission_{USE}_v{PART}.csv',index=False)

In [27]:
#np.save(f'/raid/Kaggle/otto/pred17/pred{X_VER}_v{PART}',oof)
#np.save(f'/raid/Kaggle/otto/pred15/pred{L_VER}_v{PART}',oof2)

In [28]:
sub2.head()

,session_type,labels
0,12899779_orders,59625 731692 448688 1790770 25820 475447 17002...
1,12899780_orders,1142000 582732 736515 973453 1758603 487136 59...
2,12899781_orders,918667 199008 1681537 1836671 1628918 759436 1...
3,12899782_orders,1711180 127404 562753 1344773 829180 740494 98...
4,12899783_orders,1817895 1811433 1729553 573058 1476106 1216820...
